In [2]:
import pytz
import requests
import pandas as pd
from bs4 import BeautifulSoup
# from playwright.async_api import async_playwright
from datetime import datetime, date, timedelta

In [3]:
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

---

Scrape metadata of reservoirs serving California from https://cdec.water.ca.gov/reportapp/javareports?name=ResInfo

In [4]:
url = "https://cdec.water.ca.gov/reportapp/javareports?name=ResInfo"

In [5]:
page = requests.get(url, verify=False)

In [6]:
soup = BeautifulSoup(page.content, "html.parser")

In [34]:
table = soup.find(id='RESINFO_LIST')

In [35]:
def safetxt(element):
    v = element.text.strip()
    v = v.replace("\u200b", "")
    return v

In [36]:
def safenumber(element):
    v = safetxt(element)
    v = v.replace(",", "")
    v = v.replace(" ", "")
    return v

In [37]:
def parse_table(soup):
    #tbody = soup.tbody
    row_list = table.find_all("tr")[1:]
    dict_list = []
    for row in row_list:
        cell_list = row.find_all("td")
        d = dict(
            id=safetxt(cell_list[0]),
            dam=safetxt(cell_list[1]),
            lake=safetxt(cell_list[2]),
            stream=safetxt(cell_list[3]),
            capacity=safenumber(cell_list[4]),
        )
        dict_list.append(d)
    df = pd.DataFrame(dict_list)
    df["date"] = latest_date
    return df

In [38]:
row_list = table.find_all("tr")[2:]
dict_list = []
for row in row_list:
    cell_list = row.find_all("td")
    d = dict(
        id=safetxt(cell_list[0]),
        name=safetxt(cell_list[1]),
        lake=safetxt(cell_list[2]),
        stream=safetxt(cell_list[3]),
        capacity=safenumber(cell_list[4]),
    )
    dict_list.append(d)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [12]:
metadata_df = pd.DataFrame(dict_list)

In [32]:
metadata_df

,id,name,lake,stream,capacity,url
0,APN,Alpine,Alpine Lake,Lagunitas Creek,8892,https://cdec.water.ca.gov/dynamicapp/staMeta?s...
1,ANT,Antelope,Antelope,Indian Creek,22566,https://cdec.water.ca.gov/dynamicapp/staMeta?s...
2,AST,Austrian,Lake Austrian,Los Gatos Creek,6200,https://cdec.water.ca.gov/dynamicapp/staMeta?s...
3,BRT,Barrett,Barrett Lake,Cottonwood Creek,34805,https://cdec.water.ca.gov/dynamicapp/staMeta?s...
4,BAR,Bear,Bear Reservoir,Bear Creek,7700,https://cdec.water.ca.gov/dynamicapp/staMeta?s...
...,...,...,...,...,...,...
223,VLP,Villa Park,Villa Park Reservoir,Santiago Creek,15600,https://cdec.water.ca.gov/dynamicapp/staMeta?s...
224,WRS,Warm Springs,Lake Sonoma,Russian River,381000,https://cdec.water.ca.gov/dynamicapp/staMeta?s...
225,WHR,Whale Rock,Whale Rock Lake,Old Creek,38967,https://cdec.water.ca.gov/dynamicapp/staMeta?s...
226,WHI,Whiskeytown,Whiskeytown Lake,Clear Creek,241100,https://cdec.water.ca.gov/dynamicapp/staMeta?s...


In [13]:
metadata_df["url"] = "https://cdec.water.ca.gov/dynamicapp/staMeta?station_id=" + metadata_df["id"]

In [14]:
metadata_df.to_csv("../data/metadata/reservoirs-metadata.csv", index=False)

---
Scrape detail pages for each reservoir

In [15]:
reservoir_urls = list(metadata_df.url)

In [16]:
dict_list = []
for url in reservoir_urls:
    page = requests.get(url, verify=False)
    soup = BeautifulSoup(page.content, "html.parser")
    # lol
    table = soup.find(border='1')
    try:
        cell_list = table.find_all("td")
        print(safetxt(cell_list[1]))        
        d = dict(
            id=safetxt(cell_list[1]),
            elevation=safenumber(cell_list[3]),
            basin=safetxt(cell_list[5]),
            county=safetxt(cell_list[7]),
            hydrologic_region=safetxt(cell_list[9]),
            nearby_city=safetxt(cell_list[11]),
            lat=safetxt(cell_list[13]),
            lon=safetxt(cell_list[15]),
            operator=safetxt(cell_list[17]),
            maintenance=safetxt(cell_list[19]),
        )
        dict_list.append(d)
    
    except AttributeError:
        pass

APN
ANT
AST
BRT
BAR
BRV
BRD
BTH
BLB
BOC
BMP
BQC
BWN
BWS
BDP
BIO
BHC
BUC
BIL
BCL
BTV
CCH
CVE
CRO
CMN
CMI
CFW
ALM
CPL
CSI
CAS
SLW
CHB
CHV
JNN
CLK
CLA
CLC
CGS
CMB
HNN
CTG
CYC
COY
CNV
CUY
MHV
DLV
DMV
DNP
DON
DNN
DNL
DRE
EPK
ELC
ENR
EJC
ENG
FRM
FLR
FOL
FRL
FMD
FRD
MIL
GLK
GBR
GLL
GBL
PWL
GDW
GNT
DAV
GDR
HWE
HNS
HID
MEA
HNT
ICH
INP
INL
MMW
INV
IRC
ISB
JCK
ATN
LNG
JML
JNC
KNT
KKR
KES
LGR
LFY
LGT
LEA
LKF
HMT
HDG
LVD
THC
LVY
LRA
LEW
LGV
LRK
CRW
LON
LOP
LBS
LVQ
LWB
CRY
HHL
LYS
SWB
MPL
MAR
EDN
MRT
MAT
MHW
MCO
MCS
MMR
MDO
BER
MOR
MRR
NCM
NAT
BUL
MCR
EXC
NHG
NML
SPM
NWL
NCA
ONF
HTH
OLH
ORO
LOT
OWN
PAR
HVS
PRR
PNF
PT6
PT7
BIT
PVP
PRA
PRS
PYM
QUL
RLC
RDN
RLF
RLL
RBL
RTD
SDB
SLN
SLS
SNN
SAT
SGB
LUS
SLF
SNL
SPB
SVT
PRU
SGC
LPY
SCD
SFL
SVO
SHA
SHV
SIV
SKN
SLB
SLC
JNK
SOL
SLJ
SKR
SLK
SPG
SPC
STP
SWV
SEC
STG
SCC
STD
SW3
TRM
TAB
THD
TFR
TMT
TNM
CLE
TUL
TLC
TWT
UNV
KLM
SJT
USL
UTI
UVA
VIL
VAR
TAE
EDS
VLP
WRS
WHR
WHI
WSN


In [17]:
len(dict_list)

228

In [18]:
df_list = []
for d in dict_list:
    df = pd.DataFrame(d, index=[0])
    df_list.append(df)

In [19]:
df_details = pd.concat(df_list)

In [20]:
df_details.to_csv("../data/metadata/reservoirs-details.csv", index=False)

---
### Merge

In [21]:
merge = pd.merge(metadata_df, df_details, how="left", on="id")

In [39]:
len(merge)

228

In [24]:
merge.to_csv("../data/metadata/reservoirs-metadata-details.csv", index=False)